In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time
import pickle

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def print_dict(d):
    print("{:<15} {:<15} {:<10}".format('NeuralNetwork','MEAN','STD'))
    for k, v in d.items():
        print("{:<15} {:<15} {:<10}".format(k, round(v['MEAN'], 4), round(v['STD'], 4)))

In [3]:
df = pd.read_csv('fifa19.csv')

In [4]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 
                 'Name', 'Nationality', 'Work Rate', 'Position'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [5]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [6]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score = (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [7]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [8]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [9]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

In [10]:
labelencoder_X = LabelEncoder()

df['Club'] = labelencoder_X.fit_transform(df['Club'])

Mescolo le righe del dataset

In [11]:
df_flt = df.sample(frac=1, random_state=42)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [12]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [13]:
X_train.iloc[:,55:].describe()

,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
count,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,13887.000000,1.289700e+04
mean,65.157053,46.578527,55.570966,46.430691,49.430187,52.900122,48.072946,58.135594,46.972348,47.465903,45.442500,16.716930,16.510117,16.349464,16.471304,16.782386,3.070169e+06
std,12.508469,19.003828,17.299219,20.554924,19.275723,13.794667,15.526886,11.092516,19.797010,21.520326,21.134634,17.727246,16.948568,16.567237,17.043383,17.981223,4.759708e+06
min,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.300000e+04
25%,58.000000,32.000000,43.000000,25.000000,38.000000,44.000000,38.000000,51.000000,30.000000,26.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,5.080000e+05
50%,66.000000,51.000000,58.000000,52.000000,55.000000,55.000000,49.000000,59.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1.100000e+06
75%,74.000000,62.000000,69.000000,64.000000,64.000000,63.000000,60.000000,66.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000,3.000000e+06
max,97.000000,89.000000,95.000000,90.000000,88.000000,86.000000,90.000000,88.000000,94.000000,90.000000,86.000000,88.000000,87.000000,85.000000,90.000000,86.000000,3.040000e+07


In [14]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [15]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

scaler_test = StandardScaler().fit(y_test_full)
y_test_scaled = scaler_test.transform(y_test_full) 

In [16]:
data = {
    'X_scaled': X_scaled,
    'y_scaled': y_scaled,
    'X_test_scaled': X_test_scaled,
    'y_test_scaled': y_test_scaled,
    'y_test_full': y_test_full,
}

with open('dataset_full_labels', 'wb') as f:
    pickle.dump(data, f)

In [17]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [18]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [19]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
    return round(res / tot, 2)

In [20]:
def mean_abs_error(prediction, target):
    mean = np.mean([abs(t[0] - p[0]) for t, p in zip(target, prediction) if not np.isnan(p[0])])
    return round(mean, 2)

# Cross-validation
### K-Fold

In [21]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_kfold = KFold(n_splits=3, random_state=42, shuffle=False)
inner_kfold = KFold(n_splits=5, random_state=19, shuffle=False )

In [22]:
def build_nn(input_layer, n1, n2):
    model = Sequential()

    model.add(Dense(n1, input_dim = input_layer, activation='relu'))
    
    model.add(Dense(n2, activation='relu'))

    model.add(Dense(1, activation = 'linear'))
    
    return model

In [23]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
total_results = []

for fold, (train_index_outer, test_index_outer) in enumerate(outer_kfold.split(X_scaled)):
    
    print('OUTER FOLD {}'.format(fold))
    
    max_score = -1
    optimal_neurons = -1
    
    X_train_outer, X_test_outer = X_scaled[train_index_outer], X_scaled[test_index_outer]
    y_train_outer, y_test_outer = y_scaled[train_index_outer], y_scaled[test_index_outer]
    
#     inner_mean_scores = []

    neurons = range(10, 50, 10)
    
    for n1 in neurons:
        
        for n2 in neurons:
        
            print('NEURONS {}, {}'.format(n1, n2))

            inner_scores = []

            for train_index_inner, test_index_inner in inner_kfold.split(X_train_outer):
                X_train_inner, X_test_inner = X_train_outer[train_index_inner], X_train_outer[test_index_inner]
                y_train_inner, y_test_inner = y_train_outer[train_index_inner], y_train_outer[test_index_inner]

                nn = build_nn(X_train_inner.shape[1], n1, n2)
                nn.compile(optimizer=opt, loss='mean_absolute_error', metrics=[coeff_determination])
                nn.fit(X_train_inner, y_train_inner, batch_size=125, epochs=100, shuffle=True, verbose=1)

                res = nn.predict(X_test_inner)
                loss, acc = nn.evaluate(X_train_inner, y_train_inner, verbose=1)

                reversed_res = scaler_train.inverse_transform(res)
                reversed_label = scaler_train.inverse_transform(y_test_inner)
    #             inner_scores.append(
    #                 {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
    #             )
                inner_scores.append(acc)

                print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))

            current_score = np.mean(inner_scores)

            if max_score < current_score:
                max_score = current_score
                optimal_neurons = (n1, n2)

    #         inner_mean_scores.append({'n_{}'.format(neuron): inner_scores})

    total_results.append({'fold_{}'.format(fold): (max_score, optimal_neurons)})
#     total_results.append({'fold_{}'.format(fold): inner_mean_scores})

OUTER FOLD 0
NEURONS 10, 10
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
7428/7428 [==============================] - 2s 274us/step - loss: 0.5802 - coeff_determination: -0.0724
Epoch 2/100
7428/7428 [==============================] - 0s 13us/step - loss: 0.4132 - coeff_determination: 0.4433
Epoch 3/100
7428/7428 [==============================] - 0s 13us/step - loss: 0.2776 - coeff_determination: 0.7700
Epoch 4/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.2150 - coeff_determination: 0.8550
Epoch 5/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.1818 - coeff_determination: 0.8892
Epoch 6/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.1626 - coeff_determination: 0.9082
Epoch 7/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.1474 - coeff_determination: 0.9220
Epoch 8/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.1358 - coeff_determination: 0.9284
Ep

7428/7428 [==============================] - 0s 11us/step - loss: 0.0646 - coeff_determination: 0.9778
Epoch 40/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0633 - coeff_determination: 0.9778
Epoch 41/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0632 - coeff_determination: 0.9791
Epoch 42/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0628 - coeff_determination: 0.9790
Epoch 43/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0622 - coeff_determination: 0.9790
Epoch 44/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0615 - coeff_determination: 0.9796
Epoch 45/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0609 - coeff_determination: 0.9797
Epoch 46/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0607 - coeff_determination: 0.9801
Epoch 47/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0600 - co

7428/7428 [==============================] - 0s 11us/step - loss: 0.0550 - coeff_determination: 0.9832
Epoch 80/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0539 - coeff_determination: 0.9833
Epoch 81/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0528 - coeff_determination: 0.9835
Epoch 82/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0527 - coeff_determination: 0.9843
Epoch 83/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0536 - coeff_determination: 0.9844
Epoch 84/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0523 - coeff_determination: 0.9851
Epoch 85/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0521 - coeff_determination: 0.9851
Epoch 86/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0525 - coeff_determination: 0.9852
Epoch 87/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0525 - 

7428/7428 [==============================] - 0s 12us/step - loss: 0.0793 - coeff_determination: 0.9598
Epoch 20/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0780 - coeff_determination: 0.9615
Epoch 21/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.0795 - coeff_determination: 0.9611
Epoch 22/100
7428/7428 [==============================] - 0s 13us/step - loss: 0.0769 - coeff_determination: 0.9614
Epoch 23/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0760 - coeff_determination: 0.9616
Epoch 24/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0753 - coeff_determination: 0.9622
Epoch 25/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0755 - coeff_determination: 0.9625
Epoch 26/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0743 - coeff_determination: 0.9649
Epoch 27/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0731

Epoch 59/100
7428/7428 [==============================] - 0s 13us/step - loss: 0.0548 - coeff_determination: 0.9840
Epoch 60/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.0535 - coeff_determination: 0.9851
Epoch 61/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.0538 - coeff_determination: 0.9847
Epoch 62/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.0525 - coeff_determination: 0.9853
Epoch 63/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0548 - coeff_determination: 0.9845
Epoch 64/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0519 - coeff_determination: 0.9859
Epoch 65/100
7428/7428 [==============================] - 0s 11us/step - loss: 0.0519 - coeff_determination: 0.9860
Epoch 66/100
7428/7428 [==============================] - 0s 12us/step - loss: 0.0528 - coeff_determination: 0.9856
Epoch 67/100
7428/7428 [==============================] - 0s 11us/step -

...

7429/7429 [==============================] - 1s 103us/step - loss: 0.0422 - coeff_determination: 0.9912
Epoch 69/100
7429/7429 [==============================] - 1s 94us/step - loss: 0.0433 - coeff_determination: 0.9909
Epoch 70/100
7429/7429 [==============================] - 1s 92us/step - loss: 0.0418 - coeff_determination: 0.9909
Epoch 71/100
7429/7429 [==============================] - 1s 93us/step - loss: 0.0428 - coeff_determination: 0.9911
Epoch 72/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0423 - coeff_determination: 0.9912
Epoch 73/100
7429/7429 [==============================] - 1s 84us/step - loss: 0.0416 - coeff_determination: 0.9915
Epoch 74/100
7429/7429 [==============================] - 1s 85us/step - loss: 0.0415 - coeff_determination: 0.9918
Epoch 75/100
7429/7429 [==============================] - 1s 92us/step - loss: 0.0403 - coeff_determination: 0.9920
Epoch 76/100
7429/7429 [==============================] - 1s 91us/step - loss: 0.042

7429/7429 [==============================] - 1s 96us/step - loss: 0.0816 - coeff_determination: 0.9610
Epoch 8/100
7429/7429 [==============================] - 1s 90us/step - loss: 0.0789 - coeff_determination: 0.9622
Epoch 9/100
7429/7429 [==============================] - 1s 83us/step - loss: 0.0774 - coeff_determination: 0.9640
Epoch 10/100
7429/7429 [==============================] - 1s 86us/step - loss: 0.0769 - coeff_determination: 0.9635
Epoch 11/100
7429/7429 [==============================] - 1s 84us/step - loss: 0.0730 - coeff_determination: 0.9659
Epoch 12/100
7429/7429 [==============================] - 1s 93us/step - loss: 0.0745 - coeff_determination: 0.9655
Epoch 13/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0692 - coeff_determination: 0.9697
Epoch 14/100
7429/7429 [==============================] - 1s 83us/step - loss: 0.0676 - coeff_determination: 0.9702
Epoch 15/100
7429/7429 [==============================] - 1s 85us/step - loss: 0.0678 -

7429/7429 [==============================] - 1s 88us/step - loss: 0.0430 - coeff_determination: 0.9917
Epoch 48/100
7429/7429 [==============================] - 1s 90us/step - loss: 0.0415 - coeff_determination: 0.9923
Epoch 49/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0397 - coeff_determination: 0.9933
Epoch 50/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0404 - coeff_determination: 0.9930
Epoch 51/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0414 - coeff_determination: 0.9925
Epoch 52/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0423 - coeff_determination: 0.9924
Epoch 53/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0395 - coeff_determination: 0.9933
Epoch 54/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0389 - coeff_determination: 0.9934
Epoch 55/100
7429/7429 [==============================] - 1s 89us/step - loss: 0.0399

7429/7429 [==============================] - 1s 88us/step - loss: 0.0658 - coeff_determination: 0.9713
Epoch 18/100
7429/7429 [==============================] - 1s 89us/step - loss: 0.0646 - coeff_determination: 0.9720
Epoch 19/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0640 - coeff_determination: 0.9729
Epoch 20/100
7429/7429 [==============================] - 1s 86us/step - loss: 0.0644 - coeff_determination: 0.9748
Epoch 21/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0635 - coeff_determination: 0.9750
Epoch 22/100
7429/7429 [==============================] - 1s 86us/step - loss: 0.0637 - coeff_determination: 0.9755
Epoch 23/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0605 - coeff_determination: 0.9757
Epoch 24/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0592 - coeff_determination: 0.9780
Epoch 25/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0584

7429/7429 [==============================] - 1s 98us/step - loss: 0.0367 - coeff_determination: 0.9945
Epoch 58/100
7429/7429 [==============================] - 1s 86us/step - loss: 0.0368 - coeff_determination: 0.9945
Epoch 59/100
7429/7429 [==============================] - 1s 85us/step - loss: 0.0378 - coeff_determination: 0.9943
Epoch 60/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0364 - coeff_determination: 0.9945
Epoch 61/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0374 - coeff_determination: 0.9945
Epoch 62/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0360 - coeff_determination: 0.9946
Epoch 63/100
7429/7429 [==============================] - 1s 87us/step - loss: 0.0373 - coeff_determination: 0.9945
Epoch 64/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0376 - coeff_determination: 0.9944
Epoch 65/100
7429/7429 [==============================] - 1s 88us/step - loss: 0.0375

In [25]:
total_results

[{'fold_0': (0.9950629898021475, (40, 40))},
 {'fold_1': (0.9947455669579226, (40, 40))},
 {'fold_2': (0.9949591303718153, (40, 40))}]

In [25]:
nn = build_nn(X_scaled.shape[1], 40, 40)
nn.compile(optimizer=opt, loss='mean_absolute_error', metrics=[coeff_determination])
nn.fit(X_scaled, y_scaled, batch_size=125, epochs=100, shuffle=True, verbose=1)

res = nn.predict(X_test_scaled)

scaler_test = StandardScaler().fit(y_test.values)
res = scaler_test.inverse_transform(res)

mean_abs_error(res, y_test.values), max(y_test.values), round(np.mean(y_test.values), 2), round(r2_score(y_test.values, res), 4)

Epoch 1/100
13928/13928 [==============================] - 0s 23us/step - loss: 0.2064 - coeff_determination: 0.8088
Epoch 2/100
13928/13928 [==============================] - 0s 10us/step - loss: 0.1096 - coeff_determination: 0.9467
Epoch 3/100
13928/13928 [==============================] - 0s 10us/step - loss: 0.0951 - coeff_determination: 0.9550
Epoch 4/100
13928/13928 [==============================] - 0s 10us/step - loss: 0.0877 - coeff_determination: 0.9577
Epoch 5/100
13928/13928 [==============================] - 0s 10us/step - loss: 0.0829 - coeff_determination: 0.9596
Epoch 6/100
13928/13928 [==============================] - 0s 10us/step - loss: 0.0800 - coeff_determination: 0.9627
Epoch 7/100
13928/13928 [==============================] - 0s 9us/step - loss: 0.0772 - coeff_determination: 0.9650
Epoch 8/100
13928/13928 [==============================] - 0s 10us/step - loss: 0.0732 - coeff_determination: 0.9675
Epoch 9/100
13928/13928 [==============================] - 0s 9us

(114933.19, array([13500000.]), 1638956.36, 0.9908)